In [6]:
# save as: data/processed/clean_data.py
import pandas as pd
import numpy as np
from datetime import datetime

def clean_ecommerce_data(input_path, output_path):
    print("Loading raw data...")
    df = pd.read_csv(input_path)
    
    # Create a copy to avoid SettingWithCopyWarning
    df_cleaned = df.copy()
    
    print(f"Original shape: {df_cleaned.shape}")
    
    # Step 1: Handle missing values
    print("\n1. Handling missing values...")
   
    df_cleaned['traffic_source'].fillna('Unknown', inplace=True)
    
    # Step 2: Remove duplicates
    print("\n2. Removing duplicates...")
    df_cleaned.drop_duplicates(inplace=True)
    print(f"Shape after deduplication: {df_cleaned.shape}")
    
    # Step 3: Data type conversion
    print("\n3. Converting data types...")
    df_cleaned['order_date'] = pd.to_datetime(df_cleaned['order_date'])
    df_cleaned['order_id'] = df_cleaned['order_id'].astype(str)
    df_cleaned['customer_id'] = df_cleaned['customer_id'].astype(str)
    
    # Step 4: Feature Engineering
    print("\n4. Engineering new features...")
    
    # Revenue metrics
    df_cleaned['total_price'] = df_cleaned['unit_price'] * df_cleaned['quantity']
    
    # Time-based features
    df_cleaned['order_year'] = df_cleaned['order_date'].dt.year
    df_cleaned['order_month'] = df_cleaned['order_date'].dt.month
    df_cleaned['order_quarter'] = df_cleaned['order_date'].dt.quarter
    df_cleaned['order_month_name'] = df_cleaned['order_date'].dt.month_name()
    df_cleaned['order_weekday'] = df_cleaned['order_date'].dt.day_name()
    
    # Customer metrics
    df_cleaned['is_high_value'] = df_cleaned['total_price'] > df_cleaned['total_price'].quantile(0.8)
    
    # Step 5: Remove outliers (optional but recommended)
    print("\n5. Handling outliers...")
    # Remove orders with unrealistic quantities (>50)
    outlier_mask = df_cleaned['quantity'] <= 50
    df_cleaned = df_cleaned[outlier_mask]
    print(f"Shape after outlier removal: {df_cleaned.shape}")
    
    # Step 6: Validate data integrity
    print("\n6. Validating data...")
    assert df_cleaned['order_id'].is_unique, "Order IDs should be unique"
    assert df_cleaned['total_price'].min() >= 0, "Revenue cannot be negative"
    assert df_cleaned['order_date'].min() >= pd.Timestamp('2022-01-01'), "Date out of range"
    
    # Step 7: Save cleaned data
    print("\n7. Saving cleaned data...")
    df_cleaned.to_csv(output_path, index=False)
    
    print(f"\n✅ Cleaning complete! Final shape: {df_cleaned.shape}")
    print(f"✅ Saved to: {output_path}")
    
    return df_cleaned

# Execute the cleaning
if __name__ == "__main__":
    INPUT_PATH = r"C:\Users\HP\Desktop\ecommerce_dashboard_project\raw_data\raw_data.csv"
    OUTPUT_PATH = r"C:\Users\HP\Desktop\ecommerce_dashboard_project\cleaned_data\cleaned.csv"
    
    cleaned_df = clean_ecommerce_data(INPUT_PATH, OUTPUT_PATH)
    
    # Display summary
    print("\n=== CLEANED DATA SUMMARY ===")
    print(cleaned_df.info())
    print("\nMissing values after cleaning:")
    print(cleaned_df.isnull().sum())

Loading raw data...
Original shape: (50000, 10)

1. Handling missing values...

2. Removing duplicates...
Shape after deduplication: (50000, 10)

3. Converting data types...

4. Engineering new features...

5. Handling outliers...
Shape after outlier removal: (50000, 16)

6. Validating data...

7. Saving cleaned data...


C:\Users\HP\AppData\Local\Temp\ipykernel_13756\2752910925.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['traffic_source'].fillna('Unknown', inplace=True)



✅ Cleaning complete! Final shape: (50000, 16)
✅ Saved to: C:\Users\HP\Desktop\ecommerce_dashboard_project\cleaned_data\cleaned.csv

=== CLEANED DATA SUMMARY ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          50000 non-null  object        
 1   order_date        50000 non-null  datetime64[ns]
 2   customer_id       50000 non-null  object        
 3   product_category  50000 non-null  object        
 4   product_name      50000 non-null  object        
 5   quantity          50000 non-null  int64         
 6   unit_price        50000 non-null  int64         
 7   customer_country  50000 non-null  object        
 8   traffic_source    50000 non-null  object        
 9   total_price       50000 non-null  int64         
 10  order_year        50000 non-null  int32         
 11  order_month       5000

In [10]:
import pandas as pd

# Load cleaned data
df = pd.read_csv(r"C:\Users\HP\Desktop\ecommerce_dashboard_project\cleaned_data\cleaned.csv")
df['order_date'] = pd.to_datetime(df['order_date'])

# 1. Daily aggregated data
print("Creating daily aggregates...")
daily_agg = df.groupby('order_date').agg({
    'order_id': 'count',
    'total_price': 'sum',
    'customer_id': 'nunique'
}).rename(columns={
    'order_id': 'orders',
    'total_price': 'revenue',
    'customer_id': 'unique_customers'
}).reset_index()

daily_agg['aov'] = daily_agg['revenue'] / daily_agg['orders']
daily_agg.to_csv(r"C:\Users\HP\Desktop\ecommerce_dashboard_project\cleaned_data/daily_metrics.csv", index=False)

# 2. Monthly aggregated data
print("Creating monthly aggregates...")
monthly_agg = df.groupby(['order_year', 'order_month']).agg({
    'order_id': 'count',
    'total_price': 'sum',
    'customer_id': 'nunique'
}).rename(columns={
    'order_id': 'orders',
    'total_price': 'revenue',
    'customer_id': 'unique_customers'
}).reset_index()

monthly_agg['aov'] = monthly_agg['revenue'] / monthly_agg['orders']
monthly_agg.to_csv(r"C:\Users\HP\Desktop\ecommerce_dashboard_project\cleaned_data/monthly_metrics.csv", index=False)

# 3. Product performance
print("Creating product performance...")
product_perf = df.groupby(['product_category', 'product_name']).agg({
    'order_id': 'count',
    'total_price': 'sum',
    'quantity': 'sum'
}).rename(columns={
    'order_id': 'orders',
    'total_price': 'revenue',
    'quantity': 'units_sold'
}).reset_index()

product_perf.to_csv(r"C:\Users\HP\Desktop\ecommerce_dashboard_project\cleaned_data/product_performance.csv", index=False)

print("✅ All master files created successfully!")

Creating daily aggregates...
Creating monthly aggregates...
Creating product performance...
✅ All master files created successfully!
